df_definitons  --> defintions.csv

df_1000 --> 1000.csv

df_1000_plus --> 1000_plus.csv

df_nf --> nf.csv

df_nf_post_plural --> nf_post_plural.csv

In [4]:
import pandas as pd
from lib.connect_db import get_engine
from lib.print_error import print_error
from nltk.corpus import stopwords
import inflect

p = inflect.engine()

In [5]:
df_definitons = pd.read_csv("db/definitions.csv", names=["word","definition"], sep=":", index_col=None, keep_default_na=False, na_values=[""])
df_1000 = None
df_1000_plus = None
df_nf = None
df_nf_post_plural = None

#### tasks

+ extract 1000 rows from df_definitions
+ save to 1000.csv,
+ initialize df_1000